# Assigment 3

In [227]:
import igl
import numpy as np
import meshplot as mp
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

In [228]:
v, f = igl.read_triangle_mesh("data/cow.off")
mp.plot(v, f, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Vertex normal

In [229]:
#Standard face normal
def standard_face_normals(v, f):
    return igl.per_face_normals(v, f)

#Standard vertex normal
def standard_vertex_normals(v, f):
    return igl.per_vertex_normals(v, f)

In [230]:
#Area-weighted face normal
# def area_weighted_face_normals(v, f):
#     return igl.per_face_normals(v, f, weighting_type=igl.PER_FACE_NORMALS_WEIGHTING_TYPE_AREA)

#Area-weighted vertex normal
def area_weighted_normals(v, f):
    return igl.per_vertex_normals(v, f, weighting=igl.PER_VERTEX_NORMALS_WEIGHTING_TYPE_AREA)

n = area_weighted_normals(v, f)
mp.plot(v, f,  n=n, shading={"flat": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [231]:
#Mean-curvature normal
def mean_curvature_normals(v, f, eps):
    L = igl.cotmatrix(v, f) # L is the cotangent laplacian matrix 
    M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI) # M is the mass matrix
    inv_M = diags(1 / M.diagonal()) # inv_M is the inverse of the mass matrix

    standard_normals = standard_vertex_normals(v, f)

    mean_curvature_normals = -inv_M @ L @ v # mean_curvature_normals is the mean curvature normal
    magnitude = np.linalg.norm(mean_curvature_normals, axis=1) 
    
    normals = np.zeros_like(v)
    for i in range(len(v)):
        if np.linalg.norm(mean_curvature_normals[i]) > eps:
            normals[i] = mean_curvature_normals[i] / magnitude[i]
        else:
            normals[i] = standard_normals[i]

    return normals
n1 = mean_curvature_normals(v, f, 1e-1)
mp.plot(v, f, n=n1, shading={'wireframe': False})

# n2 = mean_curvature_normals(v, f, 1e-18)
# mp.plot(v, f, n=n2, shading={'wireframe': False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [232]:
#PCA normal
def pca_normals(v, f, k):
    standard_normals = standard_vertex_normals(v, f)

    normals = np.zeros_like(v)
    for i in range(len(v)):
        distances = np.linalg.norm(v[i] - v, axis=1) # distance from v[i] to all other vertices
        distances[i] = np.inf # set distance to itself to infinity
        neighbor_indices = np.argpartition(distances, k)[:k] # indices of k nearest neighbors

        # PCA on the k nearest neighbors
        X_org = v[neighbor_indices]
        X_centered = X_org - np.mean(X_org, axis=0)
        covariance_matrix = np.cov(X_centered, rowvar=False)

        eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

        normals[i] = eigenvectors[:, np.argmin(eigenvalues)]

        if np.dot(normals[i], standard_normals[i]) < 0:
            normals[i] = -normals[i]

        normals[i] /= np.linalg.norm(normals[i])

    return normals

n = pca_normals(v, f, 10)
mp.plot(v, f, n, shading={"wireframe": False})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [233]:
#Quadratic fitting normal
def quadratic_fit_normal(v, f, k):
    standard_normals = standard_vertex_normals(v, f)
    # standard_normals = area_weighted_normals(v, f)
    
    qf_normals  = np.zeros_like(v)
    for i in range(len(v)):
        distances = np.linalg.norm(v[i] - v, axis=1) # distance from v[i] to all other vertices
        distances[i] = np.inf # set distance to itself to infinity
        neighbor_indices = np.argpartition(distances, k)[:k] # indices of k nearest neighbors

        # PCA on the k nearest neighbors
        X_org = v[neighbor_indices]
        X_centered = X_org - np.mean(X_org, axis=0)
        covariance_matrix = np.cov(X_centered, rowvar=False)

        eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

        pca_normal = eigenvectors[:, np.argmin(eigenvalues)]
        
        z = pca_normal
        x = eigenvectors[:, np.argmax(eigenvalues)]
        y = np.cross(z, x)

        frame = np.column_stack((x, y, z))
        local_neighbor_coords = (v[neighbor_indices] - v[i]) @ frame

        u_coords = local_neighbor_coords[:, 0]
        v_coords = local_neighbor_coords[:, 1]
        f_coords = local_neighbor_coords[:, 2]
        # print("local_neighbor_coords shape:", local_neighbor_coords.shape)
        

        # Fit quadratic polynomial (f(u,v) = a*u^2 + b*v^2 + c*u*v + d*u + e*v + g )

        A = np.column_stack((u_coords**2, v_coords**2, u_coords*v_coords, u_coords, v_coords, np.ones(len(u_coords))))
        coeffs = np.linalg.lstsq(A, f_coords, rcond=None)[0]

        a, b, c, d, e, g = coeffs

        # Compute normal at u=0, v=0 (at the vertex)
        # Tangent vectors:
        #   T_u = (1, 0, df/du) = (1, 0, 2*a*u + c*v + d) | u=0, v=0 = (1, 0, d)
        #   T_v = (0, 1, df/dv) = (0, 1, 2*b*v + c*u + e) | u=0, v=0 = (0, 1, e)
        tangent_u = np.array([1, 0, d]) # v = 0
        tangent_v = np.array([0, 1, e]) # u = 0
        local_normal = np.cross(tangent_u, tangent_v)
        local_normal /= np.linalg.norm(local_normal)

        normal = np.dot(frame, local_normal)

        if np.dot(normal, standard_normals[i]) < 0:
            normal = -normal
        
        qf_normals[i] = normal

    return qf_normals

n = quadratic_fit_normal(v, f, 10)
print("Quadratic Fitting Normals")
mp.plot(v, f, n, shading={"Wireframe": False})

n_pca = pca_normals(v, f, 10)
print("PCA Normals")
mp.plot(v, f, n_pca, shading={"Wireframe": False})

Quadratic Fitting Normals


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

PCA Normals


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

# Curvature

In [234]:
#gaussian curvature
k = igl.gaussian_curvature(v, f)
mp.plot(v, f, k, shading={"colormap": "coolwarm"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [235]:
l = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
 
minv = diags(1 / m.diagonal())
 
hn = -minv.dot(l.dot(v))
h = np.linalg.norm(hn, axis=1)
mp.plot(v, f, h, shading={"colormap": "coolwarm"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [236]:
# principal curvature
k_min, k_max, d_min, d_max = igl.principal_curvature(v, f)

h = 0.5 * (k_min + k_max)

h_norm = (h - np.min(h)) / (np.max(h) - np.min(h))

avg = igl.avg_edge_length(v, f) / 2.0

start_k_min = v + k_min * avg 
end_k_min   = v - k_min* avg  
 
start_k_max = v + k_max * avg
end_k_max   = v - k_max* avg  

p = mp.plot(v, f, h_norm, shading={"wireframe": False}, return_plot=True)
p.add_lines(start_k_min, end_k_min, shading={"line_color": "red"})
p.add_lines(start_k_max, end_k_max, shading={"line_color": "green"})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

2

# Smoothing with the Laplacian

In [237]:
V, F = igl.read_triangle_mesh("data/cow.off")
L = igl.cotmatrix(V, F)

N = igl.per_vertex_normals(V, F)*0.5+0.5
C = np.linalg.norm(N, axis=1)

M = igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_BARYCENTRIC)
Minv = diags(1 / M.diagonal())


In [238]:
# Explicit laplacian
exp_lambda = 0.000001
exp_iterations = 1000
V_explicit = V
for i in range(exp_iterations):
    lv = L @ V_explicit

    lvm = Minv.dot(lv)

    V_explicit = V_explicit + exp_lambda * lvm
    
    
print(V_explicit.shape)
mp.plot(V_explicit, F, C, shading={"colormap": "plasma"})

(2762, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.002776…

In [239]:
# Implicit laplacian
imp_lambda = 0.01
imp_iterations = 1
V_implicit = V
for i in range(imp_iterations):
    A = M - imp_lambda * L
    B = M.dot(V)

    V_implicit = spsolve(A, B)

mp.plot(V_implicit, F, C, shading={"colormap": "plasma"})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.029718…